In [1]:
import numpy as np
import pandas as pd
import pickle

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('phishing.csv', index_col='Index')
df

,UsingIP,LongURL,ShortURL,Symbol@,Redirecting//,PrefixSuffix-,SubDomains,HTTPS,DomainRegLen,Favicon,...,UsingPopupWindow,IframeRedirection,AgeofDomain,DNSRecording,WebsiteTraffic,PageRank,GoogleIndex,LinksPointingToPage,StatsReport,class
Index,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,-1
1,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,-1
2,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
3,1,0,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,0,-1,1,1,1,1
4,-1,0,-1,1,-1,-1,1,1,-1,1,...,1,1,1,1,1,-1,1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11049,1,-1,1,-1,1,1,1,1,-1,-1,...,-1,-1,1,1,-1,-1,1,1,1,1
11050,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,1,1,1,1,1,1,-1,1,-1
11051,1,-1,1,1,1,-1,1,-1,-1,1,...,1,1,1,1,1,-1,1,0,1,-1


In [3]:
df.columns

Index(['UsingIP', 'LongURL', 'ShortURL', 'Symbol@', 'Redirecting//',
       'PrefixSuffix-', 'SubDomains', 'HTTPS', 'DomainRegLen', 'Favicon',
       'NonStdPort', 'HTTPSDomainURL', 'RequestURL', 'AnchorURL',
       'LinksInScriptTags', 'ServerFormHandler', 'InfoEmail', 'AbnormalURL',
       'WebsiteForwarding', 'StatusBarCust', 'DisableRightClick',
       'UsingPopupWindow', 'IframeRedirection', 'AgeofDomain', 'DNSRecording',
       'WebsiteTraffic', 'PageRank', 'GoogleIndex', 'LinksPointingToPage',
       'StatsReport', 'class'],
      dtype='object')

In [4]:
df.drop(columns=['LinksPointingToPage', 'StatsReport', 'NonStdPort', 'GoogleIndex', 'WebsiteTraffic', 'PageRank',
                 'LinksInScriptTags', 'ServerFormHandler', 'InfoEmail', 'AbnormalURL', 'WebsiteForwarding',], inplace=True)

In [21]:
df.columns

Index(['UsingIP', 'LongURL', 'ShortURL', 'Symbol@', 'Redirecting//',
       'PrefixSuffix-', 'SubDomains', 'HTTPS', 'DomainRegLen', 'Favicon',
       'HTTPSDomainURL', 'RequestURL', 'AnchorURL', 'StatusBarCust',
       'DisableRightClick', 'UsingPopupWindow', 'IframeRedirection',
       'AgeofDomain', 'DNSRecording', 'class'],
      dtype='object')

In [5]:
df.isnull().any()

UsingIP              False
LongURL              False
ShortURL             False
Symbol@              False
Redirecting//        False
PrefixSuffix-        False
SubDomains           False
HTTPS                False
DomainRegLen         False
Favicon              False
HTTPSDomainURL       False
RequestURL           False
AnchorURL            False
StatusBarCust        False
DisableRightClick    False
UsingPopupWindow     False
IframeRedirection    False
AgeofDomain          False
DNSRecording         False
class                False
dtype: bool

In [6]:
df['class'].value_counts()

class
 1    6157
-1    4897
Name: count, dtype: int64

In [7]:
is_phishing = df[df['class'] == 1]
not_phishing = df[df['class'] == -1]

is_phishing_upsampled = resample(is_phishing, n_samples=len(not_phishing))

df_balanced = pd.concat([is_phishing_upsampled, not_phishing])
df_balanced['class'].value_counts()

class
 1    4897
-1    4897
Name: count, dtype: int64

In [8]:
x_data = df_balanced.drop(columns=['class'])
y_data = df_balanced['class']

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_data = scaler.fit_transform(x_data)

In [10]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_data = scaler.fit_transform(x_data)

In [11]:
linmodel = LinearRegression()

linmodel.fit(x_train, y_train)

LinearRegression()

In [12]:
y_pred = linmodel.predict(x_test)

y_pred_class = np.where(y_pred > 0, 1, -1)

accuracy = accuracy_score(y_test, y_pred_class)
print(f"Accuracy: {accuracy}\n")

report = classification_report(y_test, y_pred_class)
print("Classification Report:")
print(report)

Accuracy: 0.9173047473200613

Classification Report:
              precision    recall  f1-score   support

          -1       0.94      0.89      0.91       961
           1       0.90      0.94      0.92       998

    accuracy                           0.92      1959
   macro avg       0.92      0.92      0.92      1959
weighted avg       0.92      0.92      0.92      1959



In [13]:
pickle.dump(linmodel, open('lin.pkl', 'wb'))

In [14]:
logmodel = LogisticRegression(solver='newton-cg')

logmodel.fit(x_train, y_train)

LogisticRegression(solver='newton-cg')

In [15]:
y_pred = logmodel.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}\n")

report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Accuracy: 0.920877998979071

Classification Report:
              precision    recall  f1-score   support

          -1       0.94      0.90      0.92       961
           1       0.90      0.94      0.92       998

    accuracy                           0.92      1959
   macro avg       0.92      0.92      0.92      1959
weighted avg       0.92      0.92      0.92      1959



In [16]:
pickle.dump(logmodel, open('log.pkl', 'wb'))

In [17]:
param_grid = {
    'n_neighbors': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'chebyshev'],
}

knn = KNeighborsClassifier()

grid_search = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')

grid_search.fit(x_train, y_train)

params = grid_search.best_params_

print("Best Hyperparameters:", params)

Best Hyperparameters: {'metric': 'manhattan', 'n_neighbors': 15, 'weights': 'distance'}


In [18]:
knn = KNeighborsClassifier(n_neighbors=params['n_neighbors'], weights=params['weights'], metric=params['metric'])

knn.fit(x_train, y_train)

KNeighborsClassifier(metric='manhattan', n_neighbors=15, weights='distance')

In [19]:
y_pred = knn.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Accuracy: 0.9372128637059725
Classification Report:
              precision    recall  f1-score   support

          -1       0.95      0.92      0.94       961
           1       0.93      0.95      0.94       998

    accuracy                           0.94      1959
   macro avg       0.94      0.94      0.94      1959
weighted avg       0.94      0.94      0.94      1959



In [20]:
pickle.dump(knn, open('knn.pkl', 'wb'))